In [1]:
from src.models.level_kv_div import binaryTrainer, utils, network
from torch.utils.data import DataLoader


import torch

In [2]:
train_dataset = binaryTrainer.get_dataset('../data/processed/sample/train.csv', '../data/processed/sample/train.level.grading')
dev_dataset = binaryTrainer.get_dataset('../data/processed/sample/dev.csv', '../data/processed/sample/dev.level.grading')


2000 points 404000 data
mutual negihbors: 147722, one-direction neighbor: 54278, not neighbor: 202000
666 points 134532 data
mutual negihbors: 51636, one-direction neighbor: 15630, not neighbor: 67266


In [3]:
model = network.SiameseNet(network.EmbeddingNet())
learning_rate = 2e-3
batch_size = 2000
num_epoches = 100

verbose = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weight_decay = 1e-6
log_epoch = 3

In [4]:
# optimizer
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
dev_loader = DataLoader(dev_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
model = model.to(device)

In [5]:
best_avg_val_kl_loss, best_model, model = binaryTrainer.train_with_eval(train_loader, dev_loader, model, optimizer, num_epoches, log_epoch, verbose, device)

epoch [3]/[100] training loss: 0.0002 avg_val_margin_loss: 0.0021 train_accuracy:  0.76 val_accuracy:  0.83 
epoch [6]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0017 train_accuracy:  0.78 val_accuracy:  0.83 
epoch [9]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0017 train_accuracy:  0.79 val_accuracy:  0.83 
epoch [12]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0016 train_accuracy:  0.79 val_accuracy:  0.83 
epoch [15]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0014 train_accuracy:  0.80 val_accuracy:  0.83 
epoch [18]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0014 train_accuracy:  0.80 val_accuracy:  0.83 
epoch [21]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0011 train_accuracy:  0.80 val_accuracy:  0.83 
epoch [24]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0011 train_accuracy:  0.79 val_accuracy:  0.83 
epoch [27]/[100] training loss: 0.0001 avg_val_margin_loss: 0.0012 train_accuracy:  0.80 val_accuracy:  0.83 
epoch [30]/[1

In [6]:
best_avg_val_kl_loss

inf

In [7]:
criterion = binaryTrainer.TriMarginLoss(1, 3, 4, 6, 'mean')

In [8]:
output1, output2 = model(train_dataset[0][0].unsqueeze(0).to(device), train_dataset[0][1].unsqueeze(0).to(device))
loss, dist = criterion.forward(output1, output2, train_dataset[0][2])
print(loss.item(), dist.item())

0.0 18.02503204345703


In [9]:
((output1 - output2) ** 2).sum()

tensor(18.0250, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
test_dataset = binaryTrainer.get_dataset('../data/train.csv', '../data/train.level.grading')
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1024, pin_memory=True)

In [ ]:
test_loss, (test_acc, test_pred, test_gold) = binaryTrainer.val_one_epoch(test_dataloader, model, device)

In [ ]:
test_acc